In [1]:
import pandahouse as ph

In [2]:
connection_default = {'host': 'http://clickhouse.beslan.pro:8080',
                      'database':'default',
                      'user':'student', 
                      'password':'dpo_python_2020'
                     }

In [3]:
q = """
SELECT
    purchase_date,
    purchase_id,
    pur.client_id as client_id,
    YEAR(toDate(now())) - YEAR(cl.birth_date) AS client_age,
    dateDiff('day', cl.registration, toDate(now())) AS client_registration_age,
    cat.client_category as client_category,
    pr.promotion_name as promotion_name,
    pr.category_name as category_name,
    pr.partner_name as partner_name,
    clcit.client_city as client_city,
    cit.city as city,
    toInt32((pur.price * pur.quantity)) as revenue,
    toInt32(quantity) as quantity
FROM project_variant_2.purchase as pur
JOIN project_variant_2.client as cl ON pur.client_id = cl.client_id
JOIN 
    (
     SELECT 
        COUNT(DISTINCT purchase_id) as purchase_count,
        client_id,
        CASE
            WHEN purchase_count = 1 THEN 'new'
            ELSE 'old'
        END AS client_category
        FROM project_variant_2.purchase
        WHERE status = 1
        GROUP BY
         client_id
    ) as cat ON pur.client_id = cat.client_id
JOIN project_variant_2.promotion as pr ON pur.promotion_id = pr.promotion_id
LEFT JOIN
    (
    SELECT
        DISTINCT(client_id) as client_id,
        client_city
    FROM 
        project_variant_2.client as client
    LEFT JOIN project_variant_2.city as city ON client.client_city_id = city.client_city_id
    ) as clcit ON pur.client_id = clcit.client_id
LEFT JOIN 
    (SELECT city_id,  
        city
        FROM project_variant_2.city 
        GROUP BY city_id, city) AS cit
    ON pur.city_id = cit.city_id

WHERE
    status = 1
    AND purchase_date BETWEEN '2020-05-01' AND '2020-07-31'

"""

In [4]:
q_test = ph.read_clickhouse(query=q, connection=connection_default)
q_test

,purchase_date,purchase_id,client_id,client_age,client_registration_age,client_category,promotion_name,category_name,partner_name,client_city,city,revenue,quantity
0,2020-06-05,0,853,29,903,old,campaign_2,Фрукты,"ООО Ширяева, Хохлова и Тимофеева",Ярославль,Киев,150,5
1,2020-06-15,1,406,24,778,old,campaign_4,Прочее,KarpovCourses,Минск,Новгород,1150,5
2,2020-07-16,15,109,30,1104,old,campaign_4,Прочее,KarpovCourses,Щучинщина,Санкт-Петербург,440,8
3,2020-06-29,22,528,21,604,old,campaign_2,Фрукты,"ООО Ширяева, Хохлова и Тимофеева",Щучинщина,Щучинщина,390,2
4,2020-05-18,26,10,21,749,old,campaign_4,Прочее,KarpovCourses,Ярославль,Санкт-Петербург,570,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2690,2020-06-15,8239,352,22,1058,old,campaign_4,Прочее,KarpovCourses,Новгород,Ярославль,225,5
2691,2020-07-28,8241,602,23,748,old,campaign_1,Бакалея,Google,Щучинщина,Вена,780,6
2692,2020-07-15,8245,46,31,1035,old,campaign_1,Бакалея,Google,Москва,Минск,2275,5
2693,2020-06-09,8246,954,25,614,old,campaign_4,Прочее,KarpovCourses,Санкт-Петербург,Вена,3120,8
